# Estudantes Estrangeiros na Cidade de São Paulo

Neste tutorial, realizamos operações básicas para explorar a base de microdados de matrículas e filtrar uma das informações mais demandadas via Lei de Acesso à Informação: alunos estrangeiros, por nacionalidade, modalidade de ensino, idade etc. 

[Saiba mais](https://github.com/prefeiturasp/dados-educacao/blob/master/README.md) sobre os tutoriais dos dados abertos da SME.

Importante: também estamos aprendendo! Tem sugestões para melhorar os códigos? Não deixe de contribuir com este repositório :)

## Passo 1: Importando os dados

In [2]:
import pandas as pd
import numpy as np
import pandas_profiling as pp

Importar o arquivo .csv disponível no Portal de Dados Abertos, já descompactado (a última série de microdados disponível):

In [3]:
df_2016 = pd.read_csv('Microdados_EOL_Matriculas_2016.csv', sep=';', encoding='ISO-8859-1', low_memory=False)

## Passo 2: Conhecendo os dados

In [4]:
df_2016.head()

#Visualiza os 5 primeiros registros da base de dados.

AN_LETIVO  CD_UNIDADE_EDUCACAO   NOME_DISTRITO  CD_SETOR   TIPO_ESCOLA  \
0       2016               306329  VILA GUILHERME      8603  CR.P.CONV      
1       2016               306310          SACOMA      6802  CR.P.CONV      
2       2016               306310          SACOMA      6802  CR.P.CONV      
3       2016               400564        PIRITUBA      6301  CEI DIRET      
4       2016               400686          GRAJAU      3001  CEI INDIR      

                NOME_ESCOLA                                               DRE  \
0           LUIZ BIASI, CON    DIRETORIA REGIONAL DE EDUCACAO JACANA/TREMEMBE   
1  NOSSA SENHORA DAS MERCES           DIRETORIA REGIONAL DE EDUCACAO IPIRANGA   
2  NOSSA SENHORA DAS MERCES           DIRETORIA REGIONAL DE EDUCACAO IPIRANGA   
3     RAQUEL ZUMBANO ALTMAN   DIRETORIA REGIONAL DE EDUCACAO PIRITUBA/JARAGUA   
4            PARQUE AMERICA  DIRETORIA REGIONAL DE EDUCACAO CAPELA DO SOCORRO   

   CD_INEP_ESCOLA SITUACAO_ESCOLA  CD_TURMA           ...            \
0      35183362.0           ATIVA   1670560           ...             
1      35181183.0           ATIVA   1670600           ...             
2      35181183.0           ATIVA   1670603           ...             
3      35129938.0           ATIVA   1670205           ...             
4      35387332.0           ATIVA   1670268           ...             

   SIT_AL_TRANSF SIT_AL_FALEC  SIT_AL_NCOM SIT_AL_EXCL SIT_AL_CES  \
0              0            0            0           0          0   
1              0            0            0           0          0   
2              0            0            0           0          0   
3              0            0            0           0          0   
4              0            0            0           0          0   

  SIT_AL_REMAN SIT_AL_RECLAS SIT_AL_ATIVO  CD_PARECER_CONCL_FIN  \
0            0             0            0                   NaN   
1            0             0            0                   NaN   
2            0             0            0                   NaN   
3            0             0            0                   NaN   
4            0             0            0                   NaN   

   DESC_PARECER_CONCL_FIN  
0                     NaN  
1                     NaN  
2                     NaN  
3                     NaN  
4                     NaN  

[5 rows x 96 columns]

In [5]:
df_2016.columns
#Visualiza o nome de cada coluna disponível

Index(['AN_LETIVO', 'CD_UNIDADE_EDUCACAO', 'NOME_DISTRITO', 'CD_SETOR',
       'TIPO_ESCOLA', 'NOME_ESCOLA', 'DRE', 'CD_INEP_ESCOLA',
       'SITUACAO_ESCOLA', 'CD_TURMA', 'CD_TURNO', 'DESC_TURNO', 'CD_SERIE',
       'DESC_SERIE', 'MODALIDADE', 'MODALIDADE_SEGMENTO', 'TURMA_ESCOL',
       'NOME_TURMA', 'HORA_IN_TURMA', 'HORA_FIM_TURMA',
       'DESC_PERIODICIDADE_TURMA', 'CD_ETAPA_ENSINO', 'DESC_ETAPA_ENSINO',
       'CD_CICLO_ENSINO', 'DESC_CICLO_ENSINO', 'CD_TIPO_TURMA',
       'DESC_TIPO_TURMA', 'CD_TIPO_PROGRAMA', 'DESC_TIPO_PROGRAMA',
       'DUR_DIA_TURMA', 'DUR_DIA_HORA', 'DUR_DIA_MIN', 'DUR_SEMAN_TURMA',
       'DUR_SEM_HORA', 'DUR_SEM_MIN', 'QTD_DIAS_SEMAN_TURMA', 'X_SEMANA',
       'SEG', 'TER', 'QUA', 'QUI', 'SEX', 'SAB', 'DOM', 'CD_ALUNO_SME',
       'CD_INEP_ALUNO', 'ANO_NASC_ALUNO', 'MES_NASC_ALUNO',
       'IDADE_ALUNO_ANO_CIVIL', 'idade_aluno_31_03', 'CD_SEXO', 'CD_RACA_COR',
       'DESC_RACA_COR', 'CD_PAIS_NASC', 'DESC_PAIS_NASC', 'CD_MUN_NASC',
       'DESC_MUN_NASC'

## Passo 3: Filtrando as colunas desejadas

A partir da lista de colunas acima, vamos selecionar algumas colunas interessantes para a análise, tais como raça/cor, sexo, tipo de escola, distrito etc. Lembrando que o [dicionário de váriáveis](http://dados.prefeitura.sp.gov.br/dataset/66218af2-c669-49bc-842c-df8fcc6364ca/resource/a59b705b-febc-4968-b406-360d4a163747/download/dicionariovariaveismicrodadosmatriculasago2017.xlsx) está disponível no Portal de Dados Abertos. 

In [32]:
nacionalidades = df_2016[['AN_LETIVO', 'CD_ALUNO_SME', 'DESC_RACA_COR', 'CD_SEXO', 'ANO_NASC_ALUNO', 'MES_NASC_ALUNO', 'TIPO_ESCOLA', 'DRE', 'NOME_DISTRITO', 'TURMA_ESCOL', 'MODALIDADE_SEGMENTO', 'DESC_SERIE', 'DESC_PAIS_NASC', 'SITUACAO_MAT']]

Aqui, vamos filtrar apenas os estudantes estrangeiros matriculados na rede. Vamos aplicar três filtros: na coluna que descreve a nacionalidade, vamos selecionar tudo o que é diferente de "Brasil". Também vamos adicionar um filtro na coluna "Turma_Escol", que separa o que é turma de atividades complementares das turmas regulares; e, por fim, filtrar apenas estudantes com status 'Concluido' na Situação de Matrícula (exclui transferidos, desistentes...)

In [33]:
estrangeiros = nacionalidades[(nacionalidades['DESC_PAIS_NASC'] != 'BRASIL') 
                              & (nacionalidades['TURMA_ESCOL'] == 'S')
                             & (nacionalidades['SITUACAO_MAT'] == 'Concluido')]

In [34]:
estrangeiros.head()
#Visualiza como ficou!

AN_LETIVO  CD_ALUNO_SME  DESC_RACA_COR CD_SEXO  ANO_NASC_ALUNO  \
11        2016       6261502  NAO DECLARADA       F          2012.0   
94        2016       6448463  NAO DECLARADA       F          2015.0   
251       2016       6296484  NAO DECLARADA       M          2000.0   
385       2016       5782857  NAO DECLARADA       F          1956.0   
407       2016       6272425  NAO DECLARADA       F          1986.0   

     MES_NASC_ALUNO   TIPO_ESCOLA  \
11              7.0  CR.P.CONV      
94              7.0  CR.P.CONV      
251             1.0  EMEF           
385             6.0  EMEF           
407             5.0  EMEF           

                                                 DRE NOME_DISTRITO  \
11            DIRETORIA REGIONAL DE EDUCACAO BUTANTA       BUTANTA   
94   DIRETORIA REGIONAL DE EDUCACAO PIRITUBA/JARAGUA      PIRITUBA   
251           DIRETORIA REGIONAL DE EDUCACAO BUTANTA   RIO PEQUENO   
385       DIRETORIA REGIONAL DE EDUCACAO SANTO AMARO     JABAQUARA   
407        DIRETORIA REGIONAL DE EDUCACAO SAO MATEUS     SAPOPEMBA   

    TURMA_ESCOL MODALIDADE_SEGMENTO     DESC_SERIE DESC_PAIS_NASC SITUACAO_MAT  
11            S              CRECHE  MINI GRUPO II          INDIA    Concluido  
94            S              CRECHE     BERCARIO I        BOLIVIA    Concluido  
251           S              EJA5A8    EJA FINAL I            NaN    Concluido  
385           S              EJA5A8    EJA FINAL I            NaN    Concluido  
407           S              EJA5A8   EJA FINAL II            NaN    Concluido

## Passo 4: Limpando a base

In [24]:
estrangeiros.shape

(7992, 12)

O código acima indica que a base resultante tem 7.992 registros. Mas será que isso significa que há essa quantidade de estrangeiros? Repare que na coluna de Nacionalidade há alguns valores que aparecem como "Nan". Isso significa que a Nacionalidade não foi cadastrada na escola, por qualquer motivo. É importante saber dessa condição, mas vamos limpar a base para seguir a análise com os dados disponíveis, excluindo as linhas que possuem "Nan":

In [25]:
estrangeiros_decl = estrangeiros.dropna(how='any', subset=['DESC_PAIS_NASC'])

In [41]:
estrangeiros_decl.shape

(4570, 12)

Com isso, passamos a 4.570 registros. Já filtramos o status 'Concluido', o que já reduz bastante os casos de duplicatas (alunos com mais de um tipo de status - por exemplo, por transferência). Mas vamos aplicar o código abaixo para eliminar casos remanescentes:

In [42]:
estrangeiros_limp = estrangeiros_decl.drop_duplicates('CD_ALUNO_SME')

In [43]:
estrangeiros_limp.shape

(4494, 12)

De fato, restaram 4.494 estudantes 'únicos' que não estão cadastrados com País de Nascimento "Brasil" e que possuem um outro País declarado.

## Passo 5: Algumas análises

Com a base pronta, é possível realizar análises, tais como ordenar, agrupar e fazer gráficos. Por exemplo, ordenar por modalidade/segmento de ensino:

In [45]:
estrangeiros_limp.groupby('MODALIDADE_SEGMENTO').CD_ALUNO_SME.count()

MODALIDADE_SEGMENTO
CONVEE       1
CRECHE     733
EDPROF      11
EJA1A4      80
EJA5A8     162
Fund1     1505
Fund2      818
MEDIO        9
MOVA        52
PRE       1123
Name: CD_ALUNO_SME, dtype: int64

Agrupar por nacionalidade e ordenar pelas nacionalidade mais frequentes:

In [57]:
por_nac = pd.pivot_table(estrangeiros_limp, values='CD_ALUNO_SME', 
                         index=['DESC_PAIS_NASC'],
                         aggfunc=np.count_nonzero).sort_values(by='CD_ALUNO_SME', ascending=False)

In [71]:
por_nac.head(15)
#Visualiza as 15 nacionalidades mais frequentes

CD_ALUNO_SME
DESC_PAIS_NASC                               
BOLIVIA                                  2514
ANGOLA                                    538
REPUBLICA DO HAITI                        239
JAPAO                                     215
PARAGUAI                                  132
PERU                                      122
ARGENTINA                                 111
CONGO                                      83
COLOMBIA                                   63
PORTUGAL                                   46
SIRIA                                      44
NIGERIA                                    42
ESPANHA                                    38
ESTADOS UNIDOS DA AMERICA (EUA)            36
OUTRA NACIONALIDADE                        32

Agrupando por Diretoria Regional de Educação:

In [91]:
por_DRE = pd.pivot_table(estrangeiros_limp, values='CD_ALUNO_SME', 
                         index='DRE',
                         aggfunc=np.count_nonzero).sort_values(by='CD_ALUNO_SME', ascending=False)

In [92]:
por_DRE

CD_ALUNO_SME
DRE                                                             
DIRETORIA REGIONAL DE EDUCACAO PENHA                        1099
DIRETORIA REGIONAL DE EDUCACAO JACANA/TREMEMBE               937
DIRETORIA REGIONAL DE EDUCACAO FREGUESIA/BRASIL...           444
DIRETORIA REGIONAL DE EDUCACAO IPIRANGA                      433
DIRETORIA REGIONAL DE EDUCACAO ITAQUERA                      404
DIRETORIA REGIONAL DE EDUCACAO SAO MIGUEL                    243
DIRETORIA REGIONAL DE EDUCACAO GUAIANASES                    216
DIRETORIA REGIONAL DE EDUCACAO PIRITUBA/JARAGUA              211
DIRETORIA REGIONAL DE EDUCACAO SAO MATEUS                    166
DIRETORIA REGIONAL DE EDUCACAO CAMPO LIMPO                   133
DIRETORIA REGIONAL DE EDUCACAO BUTANTA                        85
DIRETORIA REGIONAL DE EDUCACAO CAPELA DO SOCORRO              72
DIRETORIA REGIONAL DE EDUCACAO SANTO AMARO                    51

Aplicando a função ProfileReport da bibliote Pandas Profiling para um relatório completo da base

In [ ]:
pp.ProfileReport(estrangeiros_limp)

## Passo 6: Exportando para Excel ou CSV

Depois de trabalhar a base, você pode querer exportá-la para o Excel ou outro programa. Os códigos abaixo realizam essa operação:

In [95]:
estrangeiros_limp.to_csv('estrangeiros_2016.csv', index=False)

In [96]:
estrangeiros_limp.to_excel('estrangeiros_2016.xls', index=False)

Pronto! Estão salvos no mesmo diretório em que este arquivo de tutorial está. 